# Fastpages で Notebook を使ったポスト（和訳）
> fastpages で Jupyter notebooks を使うチュートリアル

- toc: true 
- badges: true
- comments: true
- categories: [jupyter]
- image: images/chart-preview.png

# 概要

このノートブックは [fastpages](https://github.com/fastai/fastpages) を使った notebooks のデモンストレーション用です。


`fastpages` を使えば、自身の jupyter notebooks を自分のレポジトリにの中の `_notebooks` フォルダーに保存するだけで、自動的に Jekyll がブログポストに変換してくれます!


## 前付け

Jupyter Notebook や markdown のポストの最初のセルは前付けになります。前付けは Notebook のオプションのON／OFFを切り替えます。下記のフォーマットのように記入できます:

```
# タイトル
> Awesome な要約

- toc: true
- branch: master
- badges: true
- comments: true
- author: Hamel Husain & Jeremy Howard
- categories: [fastpages, jupyter]
```

- `toc: true` の設定は自動的にテーブルコンテンツを追加します
- `badges: true` の設定は自動的に GitHub や Google Colab のリンクを追加します
- `comments: true` の設定はコメント機能を追加します。コメント機能は [utterances](https://github.com/utterance/utterances) を使います。

より詳細な情報は [front matter section](https://github.com/fastai/fastpages#front-matter-related-options) of の README から確認できます。

## Markdown のショートカット

`#hide` コメントをコードセルの一番上に記載すると **input と output の両方**を隠すことができます。

`#hide_input` は **input だけ** 非表示にすることができます。

In [ ]:
#hide_input
print('コードを非表示にするため #hide_input が使われ、この文字が表示されています')

`#collapse-hide` をコードセルにの一番上に記載すると、デフォルトで **非表示** になりますが、読者に表示させる選択肢が与えられます:

In [ ]:
#collapse-hide
import pandas as pd
import altair as alt

`#collapse-show` をコードセルにの一番上に記載すると、デフォルトで **表示** になりますが、読者に非表示にさせる選択肢が与えられます:

In [ ]:
#collapse-show
cars = 'https://vega.github.io/vega-datasets/data/cars.json'
movies = 'https://vega.github.io/vega-datasets/data/movies.json'
sp500 = 'https://vega.github.io/vega-datasets/data/sp500.csv'
stocks = 'https://vega.github.io/vega-datasets/data/stocks.csv'
flights = 'https://vega.github.io/vega-datasets/data/flights-5k.json'

## Altair を使ったインタラクティブなチャート

Altair は使ったチャートはインタラクティブに操作できます。例えばこちらの [this repo](https://github.com/uwdata/visualization-curriculum) 、 特に [this notebook](https://github.com/uwdata/visualization-curriculum/blob/master/altair_interaction.ipynb) などです。

In [ ]:
# hide
df = pd.read_json(movies) # load movies data
genres = df['Major_Genre'].unique() # get unique field values
genres = list(filter(lambda d: d is not None, genres)) # filter out None values
genres.sort() # sort alphabetically

In [ ]:
#hide
mpaa = ['G', 'PG', 'PG-13', 'R', 'NC-17', 'Not Rated']

### Example 1: ドロップダウン

In [ ]:
# single-value selection over [Major_Genre, MPAA_Rating] pairs
# use specific hard-wired values as the initial selected values
selection = alt.selection_single(
    name='Select',
    fields=['Major_Genre', 'MPAA_Rating'],
    init={'Major_Genre': 'Drama', 'MPAA_Rating': 'R'},
    bind={'Major_Genre': alt.binding_select(options=genres), 'MPAA_Rating': alt.binding_radio(options=mpaa)}
)
  
# scatter plot, modify opacity based on selection
alt.Chart(movies).mark_circle().add_selection(
    selection
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(selection, alt.value(0.75), alt.value(0.05))
)

Example 2: ツールチップ


In [ ]:
alt.Chart(movies).mark_circle().add_selection(
    alt.selection_interval(bind='scales', encodings=['x'])
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # use min extent to stabilize axis title placement
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
)

### Example 3: さらにツールチップ

In [ ]:
# select a point for which to provide details-on-demand
label = alt.selection_single(
    encodings=['x'], # limit selection to x-axis value
    on='mouseover',  # select on mouseover events
    nearest=True,    # select data point nearest the cursor
    empty='none'     # empty selection includes no data points
)

# define our base line chart of stock prices
base = alt.Chart().mark_line().encode(
    alt.X('date:T'),
    alt.Y('price:Q', scale=alt.Scale(type='log')),
    alt.Color('symbol:N')
)

alt.layer(
    base, # base line chart
    
    # add a rule mark to serve as a guide line
    alt.Chart().mark_rule(color='#aaa').encode(
        x='date:T'
    ).transform_filter(label),
    
    # add circle marks for selected time points, hide unselected points
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_selection(label),

    # add white stroked text to provide a legible background for labels
    base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
        text='price:Q'
    ).transform_filter(label),

    # add text labels for stock prices
    base.mark_text(align='left', dx=5, dy=-5).encode(
        text='price:Q'
    ).transform_filter(label),
    
    data=stocks
).properties(
    width=700,
    height=400
)

## データテーブル

普段どおりデータテーブルもブログに表示することができます:

In [ ]:
movies = 'https://vega.github.io/vega-datasets/data/movies.json'
df = pd.read_json(movies)
# display table with pandas
df[['Title', 'Worldwide_Gross', 
    'Production_Budget', 'Distributor', 'MPAA_Rating', 'IMDB_Rating', 'Rotten_Tomatoes_Rating']].head()

## 画像

### ローカルにある画像

ローカル画像を参照することができ、その画像は自動的にブログにコピーされます。下記のマークダウン記法で画像を表示することができます:

`![](my_icons/fastai_logo.png)`

![](my_icons/fastai_logo.png)

### リモートにある画像

リモートにある画像は下記のマークダウン記法で表示することができます:

`![](https://image.flaticon.com/icons/svg/36/36686.svg)`

![](https://image.flaticon.com/icons/svg/36/36686.svg)

### アニメーション Gif

アニメーション Gif も動作します!

`![](https://upload.wikimedia.org/wikipedia/commons/7/71/ChessPawnSpecialMoves.gif)`

![](https://upload.wikimedia.org/wikipedia/commons/7/71/ChessPawnSpecialMoves.gif)

### キャプション

下記のようなマークダウン記法でキャプションも追加することができます：

```
![](https://www.fast.ai/images/fastai_paper/show_batch.png "Credit: https://www.fast.ai/2020/02/13/fastai-A-Layered-API-for-Deep-Learning/")
```


![](https://www.fast.ai/images/fastai_paper/show_batch.png "Credit: https://www.fast.ai/2020/02/13/fastai-A-Layered-API-for-Deep-Learning/")

# その他

## Tweetcards

`> twitter: https://twitter.com/jakevdp/status/1204765621767901185?s=20` のように書くと下記のようになります:

> twitter: https://twitter.com/jakevdp/status/1204765621767901185?s=20

## Youtube Videos

`> youtube: https://youtu.be/XfoYk_Z5AkI` のように書くと下記のようになります:


> youtube: https://youtu.be/XfoYk_Z5AkI

## Boxes / Callouts 

`> Warning: There will be no second warning!` のように書くと下記のようになります:


> Warning: There will be no second warning!



`> Important: Pay attention! It's important.` のように書くと下記のようになります:

> Important: Pay attention! It's important.



`> Tip: This is my tip.` のように書くと下記のようになります:

> Tip: This is my tip.

`> Note: Take note of this.` のように書くと下記のようになります:

> Note: Take note of this.


`> Note: A doc link to [an example website: fast.ai](https://www.fast.ai/) should also work fine.` のように書くと下記のようになります:

> Note: A doc link to [an example website: fast.ai](https://www.fast.ai/) should also work fine.


## 脚注

脚注を書くことができますが、マークダウン記法とはことなります。 [このガイドが記法についての詳細を記載しています](https://github.com/fastai/fastpages/blob/master/_fastpages_docs/NOTEBOOK_FOOTNOTES.md)、下記のようになります:

```
{% raw %}これが脚注です {% fn 1 %}.
別の脚注です {% fn 2 %}
{{ '脚注の詳細です.' | fndetail: 1 }}
{{ '別の脚注の詳細です。[link](www.github.com)を使用することもできます!' | fndetail: 2 }}{% endraw %}
```

これが脚注です {% fn 1 %}.
別の脚注です {% fn 2 %}
{{ '脚注の詳細です.' | fndetail: 1 }}
{{ '別の脚注の詳細です。[link](www.github.com)を使用することもできます!' | fndetail: 2 }}